# Organización de población censada

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [1]:
import os

def clean_data(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['censada']        
    headers = "Serie;Codigo_municipio;2022;2023\n"

    # Limpiar y guardar los files CSV en la carpeta de salida
    for file in csv_files:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = True
                    if 'Serie' in line:
                            delete = True
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-3:]) + '\n')
                    if 'Zarzalejo' in line:
                        delete_next = True

input_folder = ".\original_data"
output_folder = ".\clean_data"

clean_data(input_folder, output_folder)


## Unión de datos por edad

In [4]:
import os
import csv
import re

def join_data(folder_path, output_path):
    # Crear un diccionario para almacenar los datos sumados
    join_data = {}

    # Obtener la lista de archivos en la carpeta (ordenado por edad)
    csv_files = sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else float('inf'))
    
    # Iterar sobre cada archivo en la carpeta
    for file in csv_files:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)

                # Obtener la extensión del nombre del file (por ejemplo, "0a4" o "5a9")
                age_gap = file.split('_')[-1].split('.')[0]
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in join_data:
                        join_data[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos de cada año para el intervalo de edad actual
                    for i, value in enumerate(row[3:5], start=2022):
                        key = f"censo_{age_gap}_{i}"
                        # Agregar los datos al diccionario enlazado
                        join_data[(serie, codigo_municipio, nombre)][key] = value if value else ''

    # Escribir los datos sumados en un nuevo file CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        final_keys = []

        for key, values in join_data.items():
            if final_keys == []:
                final_keys.extend(values.keys())
        final_keys = [str(key) for key in final_keys]
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *final_keys])

            # Escribir los datos sumados
        for key, values in join_data.items():
            line = [key[0], key[1], key[2]]
            for key in final_keys:
                line.append(values.get(key, ''))
            csv_writer.writerow(line)


folder_path = './clean_data'
output_path = '../union_final/censo_edad.csv'

join_data(folder_path, output_path)